In [1]:
'''
Notebook for Are.na crawling
'''

'''
Useful GETs:

http://api.are.na/v2/channels/arena-influences/channels



'''

'\nUseful GETs:\n\nhttp://api.are.na/v2/channels/arena-influences/channels\n\n\n\n'

In [1]:
### IMPORTS ###
if __name__ == '__main__':
    import requests
    import math
    import networkx as nx
    from threading import Thread
    from queue import Queue

In [2]:
def multi_generateChannelNetwork(startSlug, num_threads):
    def get_connected_channels_from_queue(queue,r):
        while True:
            slug = queue.get()
            slugs = getConnectedChannels(slug)
            l.append(slugs)
            for slug in slugs:
                r.put(slug)
                queue.put(slug)
            queue.task_done()
            
            
    res = Queue()
    q = Queue()
    q.put(startSlug)
    l = []
    for i in range(num_threads):
        print("hello")
        worker = Thread(target=get_connected_channels_from_queue, args=(q,res))
        worker.setDaemon(True)
        worker.start()
        q.join()
        
    return l




In [ ]:
###this works
x = requests.get('https://api.are.na/v2/channels/nynets-_sal8iqlt8y/thumb')
x

In [ ]:
x.json()['contents'][5]

In [ ]:
collab_r = requests.get('http://api.are.na/v2/channels/nynets-_sal8iqlt8y/connections')
collab_r

In [ ]:
(collab_r.json()['channels'])[0]['slug']

In [ ]:
test_linkage = requests.get('https://api.are.na/v2/channels/transfixions')
test_linkage

In [ ]:
test_linkage.json()

In [ ]:
multi_generateChannelNetwork('nynets-_sal8iqlt8y',100)

In [4]:
def getConnectedChannels(slug):
    #getConnectedChannels(slug) = [c1,c2,...]
    #slug: channel slug str
    #[c1,c2,...] where c1,c2,... is linked to the current channel
    
    GET = f'http://api.are.na/v2/channels/{slug}/connections'
    try:
        collab_json_list = requests.get(GET).json()['channels']
    except:
        return []
    slugs = [json['slug'] for json in collab_json_list]
    
    return slugs

getConnectedChannels('nynets-_sal8iqlt8y')

['random_2', 'radical-technologies-and-art']

In [5]:
def generateChannelNetwork(startSlug,depth):
    '''
    bfs for network generation 
    generateChannelNetwork(startSlug,depth) = network : networkx 
    where nodes represent channels and edges exist if one channels appear in another 
    '''
    q = Queue()
    seen = set([startSlug])
    q.put(startSlug)
    G = nx.Graph()
    
    while depth > 0:
        for i in range(q.qsize()):
            curr_slug = q.get_nowait()
            print(curr_slug)
            slugs = getConnectedChannels(curr_slug)
            for slug in slugs:
                if slug not in seen:
                    seen.add(slug)
                    q.put(slug)
                        
                    G.add_edge(curr_slug,slug)
                    
                
        depth -= 1
        
    return G

In [45]:
def multi_generateChannelNetwork(startSlugs, num_threads):
    def get_connected_channels_from_queue(queue,r):
        while True:
            slug = queue.get()
            slugs = getConnectedChannels(slug)            
            for slug in slugs:
                r.put(slug)
            queue.task_done()

    res = Queue()
    q = Queue()
    for slug in startSlugs:
        q.put(slug)
    
    for i in range(num_threads):
        worker = Thread(target=get_connected_channels_from_queue, args=(q,res))
        worker.setDaemon(True)
        worker.start()
        
    q.join()
    return list(res.queue)





In [ ]:
def 

In [47]:
l = ['nynets-_sal8iqlt8y',
'random_2',
'radical-technologies-and-art',
'research-l25g9er4i-m',
'knowledge_archive',
'reads-fdr9qkol6do',
'be_online',
'ok-computer',
'html-css-js']

r = multi_generateChannelNetwork(l,10)

print(r)

hello
nynets-_sal8iqlt8y slug
hello
random_2hello slug

radical-technologies-and-art slug
hello
research-l25g9er4i-m hello
slug
knowledge_archivehello
 reads-fdr9qkol6do slug
hello
be_onlinehello
slug 
slugok-computer
hello
 slug
html-css-js slug
hello
['random_2', 'radical-technologies-and-art', 'web-tools-gkefzoqrvxs', 'ok-computer', 'web-fp7au25baj0', 'coding-e8nhgcunnbw', 'radical-technologies-and-art', 'archive-web-evtmglkr8eg', 'web-resources-pjqxzphljxq', 'designdesign-2ykrjx-dary', 'oh-yeah-baby-oh-yeah', 'lo97gec3rmo', 'research-l25g9er4i-m', 'knowledge_archive', 'reads-fdr9qkol6do', 'be_online', 'ok-computer', 'html-css-js', 'hyper-zkykreilrus', 'thieve-s-trove', 'thesis-h4a0qg_8gsi', 'web-resources-pjqxzphljxq', 'assortment-of-channels-of-channel-assortments', 'grew-up-online', 'abolition-is', 'to-educate-myself-on-benxc_uwjwe', 'read-zvjf5hpwj6q', 'sometimes-i-just-be-thinking', 'lookback', 'the-bits-that-make-up-a-design-philosophy', 'books-v2defximxss', 'information-exper

In [37]:
r.qsize()

56

In [ ]:
nx.drawing.nx_pylab.draw(G,l)

In [ ]:
from matplotlib import pylab
import matplotlib.pyplot as plt
import networkx as nx

def save_graph(graph,file_name):
#initialze Figure
    plt.figure(num=None, figsize=(20, 20), dpi=80)
    plt.axis('off')
    fig = plt.figure(1)
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph,pos)
    nx.draw_networkx_edges(graph,pos)
    nx.draw_networkx_labels(graph,pos)

    cut = 1.00
    xmax = cut * max(xx for xx, yy in pos.values())
    ymax = cut * max(yy for xx, yy in pos.values())
    plt.xlim(-2*xmax, 2*xmax)
    plt.ylim(-2*ymax, 2*ymax)

    plt.savefig(file_name,bbox_inches="tight")
    pylab.close()
    del fig

    

In [ ]:
multi_generateChannelNetwork('nynets-_sal8iqlt8y',1)

In [ ]:
nx.algorithms.cycles.find_cycle(G)

In [ ]:
import os
os.cwd()

In [ ]:
G = nx.Graph()
G.add_node("a")
G.add_node("b")

In [ ]:
G.add_edges_from((["a"],["b"]))

In [ ]:
nx.draw_networkx(G)

In [ ]:
nx.dr